In [1]:
import codecs
import os
import json
import pickle
import tqdm
import numpy as np
from hazm import *
from itertools import chain
from random import shuffle
from scipy import sparse

In [2]:
import json

DIRNAME = '../dataset'
data = []
for i in range(1, 8):
    with open(f'{DIRNAME}/namnak-p{i}.json', 'r', encoding="utf-8") as f:
        data.extend(json.loads(f.read()))
    with open(f'{DIRNAME}/hidoctor-p{i}.json', 'r', encoding="utf-8") as f:
        data.extend(json.loads(f.read()))

In [3]:
# normalization
from hazm import *

normalizer = Normalizer()

normalized_data = []
for item in tqdm.tqdm(data):
    normalized_data.append({"title" : normalizer.normalize(item['title']), "text":normalizer.normalize(item['text']), 'link':item['link']})

100%|█████████████████████████████████████████████████████████████████████████████| 4322/4322 [00:10<00:00, 425.94it/s]


In [4]:
# Persian Stopwords
# https://github.com/sobhe/hazm/blob/master/hazm/data/stopwords.dat
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]
custom_stop_words = [normalizer.normalize(x.strip()) for x in codecs.open('custom_stopwords.txt','r','utf-8').readlines()]
total_stop_words = custom_stop_words + stopwords

In [5]:
all_doc_tokens = []
for item in tqdm.tqdm(normalized_data):
  all_doc_tokens.append([_ for _ in word_tokenize(item['text'])  if _ not in total_stop_words])

100%|█████████████████████████████████████████████████████████████████████████████| 4322/4322 [00:19<00:00, 226.52it/s]


In [6]:
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def get_lemma_set(tok, opt=1):
    if opt ==1:
        return stemmer.stem(tok)
    if opt ==2:
        return lemmatizer.lemmatize(tok)

In [7]:
opt = 2
tokens_after_lem = []

for tokens in all_doc_tokens:
    tokens_after_lem.append([get_lemma_set(token, opt) for token in tokens])

In [8]:
def create_boolean_matrix():
  all_tokens = list(set(chain.from_iterable(tokens_after_lem)))
  titles = []
  for item in normalized_data:
    titles.append(item['title'])
  matrix = np.zeros(shape=(len(normalized_data), len(all_tokens)), dtype=bool)
  for i in range(len(tokens_after_lem)):
    title = normalized_data[i]['title']
    for token in tokens_after_lem[i]:
      matrix[titles.index(title)][all_tokens.index(token)] = True
  return matrix, titles, all_tokens

In [9]:
boolean_matrix, titles, all_tokens = create_boolean_matrix()

In [10]:
boolean_matrix.shape

(4322, 43421)

In [11]:
# save matrix
os.makedirs('../models', exist_ok=True)
np.save('../models/doc-term-boolean.npy', boolean_matrix)

In [12]:
# load matrix
boolean_matrix = np.load('../models/doc-term-boolean.npy')
boolean_matrix.shape

(4322, 43421)

In [13]:
# save doc data
data = []
for i, item in enumerate(normalized_data):
    data.append({'title':item['title'], 'link':item['link']})
with open(f'../models/doc-term-boolean-data.json', 'w', encoding="utf-8") as f:
    json.dump(data, f)

In [14]:
# save all tokens
all_tokens = list(set(chain.from_iterable(tokens_after_lem)))
with open(f'../models/doc-term-boolean-tokens.json', 'w', encoding="utf-8") as f:
    json.dump(all_tokens, f)

In [20]:
class BooleanModel:
    def __init__(self):
        self.lemmatizer = Lemmatizer()
        self.boolean_matix = np.load('../models/doc-term-boolean.npy')
        with open(f'../models/doc-term-boolean-data.json', 'r', encoding="utf-8") as f:
            self.data = json.loads(f.read())
        with open(f'../models/doc-term-boolean-tokens.json', 'r', encoding="utf-8") as f:
            self.all_tokens = json.loads(f.read())

    def process_query(self, query):
        query_tokens = query.strip().lower().split()
        query_tokens = [self.lemmatizer.lemmatize(token) for token in query_tokens]
        operands = list()
        operators = list()
        i = 0
        while i < len(query_tokens):
            token = query_tokens[i]
            if token == 'and' or token == 'or':
                operators.append(token)
            elif token == 'not':
                operands.append(('not', query_tokens[i + 1]))
                i += 1
            else:
                operands.append(('', token))
            i += 1
        return operands, operators

    def get_token_column(self, token):
        if token[1] in self.all_tokens:
            column = self.boolean_matix[:, self.all_tokens.index(token[1])]
            if token[0] == 'not':
                column = ~column
            return column
        else:
            return np.zeros(len(self.data), dtype=bool)

    def operate(self, operand1, operand2, operator):
        if operator == 'and':
            return operand1 & operand2
        elif operator == 'or':
            return operand1 | operand2

    def get_nearest_neighbors(self, query, k):
        neighbors = list()
        operands, operators = self.process_query(query)
        n = len(operands)
        if n < 2:
            result = self.get_token_column(operands[0])
        else:
            column_operand_1 = self.get_token_column(operands[0])
            column_operand_2 = self.get_token_column(operands[1])
            result = self.operate(column_operand_1, column_operand_2, operators[0])
            for i, operand in enumerate(operands[2:]):
                other_operand = self.get_token_column(operand)
                result = self.operate(result, other_operand, operators[i + 1])
        indices = result.nonzero()[0]
        for index in indices:
            neighbors.append(self.data[index])
        shuffle(neighbors)
        return neighbors[:k] if k < len(neighbors) else neighbors

    def print_similars(self, query, k=10):
        ls = self.get_nearest_neighbors(query, k)
        for i, item in enumerate(ls):
            print(f"{i + 1}- title: {item['title']}")
            print(f"{i + 1}- link: {item['link']}")
            print('-------------------------')

In [21]:
BooleanModel().print_similars('مرگ')

1- title: بیماری سیاه زخم را باهم مرور میکنیم
1- link: https://www.hidoctor.ir/11833_%d8%a8%db%8c%d9%85%d8%a7%d8%b1%db%8c-%d8%b3%db%8c%d8%a7%d9%87-%d8%b2%d8%ae%d9%85-%d8%b1%d8%a7-%d8%a8%d8%a7%d9%87%d9%85-%d9%85%d8%b1%d9%88%d8%b1-%d9%85%db%8c%da%a9%d9%86%db%8c%d9%85.html/
-------------------------
2- title: اگر عمر طولانی می‌خواهید بخوانید
2- link: https://namnak.com/how-to-live-longer.p8598
-------------------------
3- title: چگونه به شخصی که قصد خودکشی دارد مشاوره دهیم؟
3- link: https://www.hidoctor.ir/357610_%da%86%da%af%d9%88%d9%86%d9%87-%d8%a8%d9%87-%d8%b4%d8%ae%d8%b5%db%8c-%da%a9%d9%87-%d9%82%d8%b5%d8%af-%d8%ae%d9%88%d8%af%da%a9%d8%b4%db%8c-%d8%af%d8%a7%d8%b1%d8%af-%d9%85%d8%b4%d8%a7%d9%88%d8%b1%d9%87.html/
-------------------------
4- title: سیگار کشیدن و اثرات آن بر روی پوست
4- link: https://www.hidoctor.ir/353082_%d8%b3%db%8c%da%af%d8%a7%d8%b1-%da%a9%d8%b4%db%8c%d8%af%d9%86-%d9%88-%d8%a7%d8%ab%d8%b1%d8%a7%d8%aa-%d8%a2%d9%86-%d8%a8%d8%b1-%d8%b1%d9%88%db%8c-%d9%be%d9%88%d8%b3%d8%

In [22]:
BooleanModel().print_similars('کرونا and ریه')

1- title: آنچه باید درباره عفونت همزمان کرونا و آنفلوانزا بدانید
1- link: https://www.hidoctor.ir/357168_%d8%a2%d9%86%da%86%d9%87-%d8%a8%d8%a7%db%8c%d8%af-%d8%af%d8%b1%d8%a8%d8%a7%d8%b1%d9%87-%d8%b9%d9%81%d9%88%d9%86%d8%aa-%d9%87%d9%85%d8%b2%d9%85%d8%a7%d9%86-%da%a9%d8%b1%d9%88%d9%86%d8%a7-%d9%88-%d8%a2.html/
-------------------------
2- title: در مورد بیماری کرونا ویروس چه می‌دانید؟
2- link: https://www.hidoctor.ir/346899_%d8%af%d8%b1-%d9%85%d9%88%d8%b1%d8%af-%d8%a8%db%8c%d9%85%d8%a7%d8%b1%db%8c-%da%a9%d8%b1%d9%88%d9%86%d8%a7-%d9%88%db%8c%d8%b1%d9%88%d8%b3-%da%86%d9%87-%d9%85%db%8c-%d8%af%d8%a7%d9%86%db%8c%d8%af%d8%9f.html/
-------------------------
3- title: علائم و عوارض بلند مدت کرونا + حقایق ناگفته
3- link: https://namnak.com/coronavirus-long-term-effects.p83159
-------------------------
4- title: علائم نشان دهنده انتشار ویروس کرونا در ریه‌ها
4- link: https://www.hidoctor.ir/352292_%d8%b9%d9%84%d8%a7%d8%a6%d9%85-%d9%86%d8%b4%d8%a7%d9%86-%d8%af%d9%87%d9%86%d8%af%d9%87-%d8%a7%d9%86%

In [23]:
BooleanModel().print_similars('کرونا and not مرگ')

1- title: ردپای کرونا در سردخانه‌های مواد غذایی چین
1- link: https://www.hidoctor.ir/350895_%d8%b1%d8%af%d9%be%d8%a7%db%8c-%da%a9%d8%b1%d9%88%d9%86%d8%a7-%d8%af%d8%b1-%d8%b3%d8%b1%d8%af%d8%ae%d8%a7%d9%86%d9%87-%d9%87%d8%a7%db%8c-%d9%85%d9%88%d8%a7%d8%af-%d8%ba%d8%b0%d8%a7%db%8c%db%8c-%da%86.html/
-------------------------
2- title: علت و روش جلوگیری از سرگیجه بعد از ورزش چیست؟
2- link: https://www.hidoctor.ir/354462_%d8%b9%d9%84%d8%aa-%d9%88-%d8%b1%d9%88%d8%b4-%d8%ac%d9%84%d9%88%da%af%db%8c%d8%b1%db%8c-%d8%a7%d8%b2-%d8%b3%d8%b1%da%af%db%8c%d8%ac%d9%87-%d8%a8%d8%b9%d8%af-%d8%a7%d8%b2-%d9%88%d8%b1%d8%b2%d8%b4.html/
-------------------------
3- title: این علائم کرونا را جدی بگیرید
3- link: https://www.hidoctor.ir/354112_%d8%a7%db%8c%d9%86-%d8%b9%d9%84%d8%a7%d8%a6%d9%85-%da%a9%d8%b1%d9%88%d9%86%d8%a7-%d8%b1%d8%a7-%d8%ac%d8%af%db%8c-%d8%a8%da%af%db%8c%d8%b1%db%8c%d8%af.html/
-------------------------
4- title: شایع‌ترین علائم پس از ابتلا به کرونا / چه زمانی باید درخواست کمک کنیم؟
4- link: h

In [24]:
BooleanModel().print_similars('ویتامین or میوه')

1- title: علل و عوامل خطر بروز پوکی استخوان
1- link: https://www.hidoctor.ir/357093_%d8%b9%d9%84%d9%84-%d9%88-%d8%b9%d9%88%d8%a7%d9%85%d9%84-%d8%ae%d8%b7%d8%b1-%d8%a8%d8%b1%d9%88%d8%b2-%d9%be%d9%88%da%a9%db%8c-%d8%a7%d8%b3%d8%aa%d8%ae%d9%88%d8%a7%d9%86.html/
-------------------------
2- title: خانم‌ها چه غذاهایی برای پیشگیری از سرطان بخورند
2- link: https://www.hidoctor.ir/62421_%d8%ae%d8%a7%d9%86%d9%85-%d9%87%d8%a7-%da%86%d9%87-%d8%ba%d8%b0%d8%a7%d9%87%d8%a7%db%8c%db%8c-%d8%a8%d8%b1%d8%a7%db%8c-%d9%be%db%8c%d8%b4%da%af%db%8c%d8%b1%db%8c-%d8%a7%d8%b2-%d8%b3%d8%b1%d8%b7%d8%a7.html/
-------------------------
3- title: آجیل در بهبود چه بیماری هایی می‌تواند موثر باشد؟
3- link: https://www.hidoctor.ir/355970_%d8%a2%d8%ac%db%8c%d9%84-%d8%af%d8%b1-%d8%a8%d9%87%d8%a8%d9%88%d8%af-%da%86%d9%87-%d8%a8%db%8c%d9%85%d8%a7%d8%b1%db%8c-%d9%87%d8%a7%db%8c%db%8c-%d9%85%db%8c-%d8%aa%d9%88%d8%a7%d9%86%d8%af-%d9%85%d9%88.html/
-------------------------
4- title: مواد مغذی و مناسب برای رشد مو
4- link: https